In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.5.2
!pip install catboost
!pip install dcor
!pip install deap


import dcor
import pandas as pd
pd.set_option('display.max_rows', 500)
import numpy as np
import os
from tqdm import tqdm
from joblib import Parallel, delayed

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, f_regression, chi2, mutual_info_classif, mutual_info_regression, VarianceThreshold
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score, mutual_info_score, accuracy_score
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Permute, Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, Dropout, LSTM
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from lightgbm.sklearn import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor, XGBClassifier
from sklearn.linear_model import Ridge, Lasso

from deap import base, creator, tools, algorithms  # Genetic Algorithm library
import random

from scipy.stats import spearmanr
from scipy.cluster.hierarchy import linkage, fcluster
import shap
from scipy.spatial.distance import correlation

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Successfully uninstalled scikit-learn-1.5.2
  Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)


In [5]:
#Import data
train = pd.read_csv("gdrive/MyDrive/Learning/Superconductivity/train.csv")
formula_train = pd.read_csv("gdrive/MyDrive/Learning/Superconductivity/formula_train.csv")
print(f"Train dataset shape: {train.shape}")
print(f"Train_formula dataset shape: {formula_train.shape}")

test = pd.read_csv("gdrive/MyDrive/Learning/Superconductivity/test.csv")
formula_test = pd.read_csv("gdrive/MyDrive/Learning/Superconductivity/formula_test.csv")
print(f"Test dataset shape: {test.shape}")
print(f"Test_formula dataset shape: {formula_test.shape}")


Train dataset shape: (17010, 82)
Train_formula dataset shape: (17010, 88)
Test dataset shape: (4253, 81)
Test_formula dataset shape: (4253, 87)


**Part 1: Using CNNs to predict temperature based on formula data**

In [7]:
#Set up architecture to train formula data on relevant column
material_elements = formula_train.drop(columns=["material"])
material_elements = material_elements.astype(float)

element_features = pd.read_csv("gdrive/MyDrive/Learning/Superconductivity/element_data.csv")
del element_features['Unnamed: 0']

In [8]:
# Initialize a list to store merged data
merged_data = []
targets = []

# Iterate through each material in material_elements with a progress bar
for material_index, material_row in tqdm(material_elements.iterrows(), total=len(material_elements), desc="Processing materials"):
    material_name = material_row.name  # Material is the index (name) of the row
    critical_temp = material_row['critical_temp']  # Get the target variable
    material_row = material_row.drop('critical_temp')  # Remove the target variable from the row
    # Get the non-zero element counts for this material
    elements_in_material = material_row[material_row > 0].index.tolist()  # List of elements in material with non-zero counts

    # Create a temporary DataFrame for the current material
    material_data = []

    for i in range(10):  # Maximum 10 rows per material
        if i < len(elements_in_material):
            element = elements_in_material[i]
            element_info = element_features[element_features['Element'] == element].copy()
            # Add element count (how many times this element occurs) to the feature data
            element_info['count'] = material_row[element]
        else:
            # Fill with zeros for padding
            element_info = pd.DataFrame(np.zeros((1, len(element_features.columns) + 1)),
                                        columns=element_features.columns.tolist() + ['count'])

        # Append the element's features to the material data
        material_data.append(element_info)

    # Combine material data into a single DataFrame for this material
    material_df = pd.concat(material_data, ignore_index=True)
    material_df['material_name'] = material_name  # Add the material name as a column

    # Append the material's data to the merged dataset
    merged_data.append(material_df)
    targets.append(critical_temp)



Processing materials: 100%|██████████| 17010/17010 [02:20<00:00, 121.25it/s]


In [9]:
# Combine all material data into a single DataFrame
features = ['count','AtomicMass','AtomicNumber','FirstIonizationEnergy','AtomicRadius','BoilingPoint',
            'Density','ElectronAffinity','Electronegativity','FusionHeat','MeltingPoint','SpecificHeat',
            'ThermalConductivity','ThermalExpansion','Valence','MolarVolume','Period', 'material_name'
]
final_df = pd.concat(merged_data, ignore_index=True)
formula_df = final_df[features]

# Group by material_name to get arrays for CNN
grouped_fixed = []
for material_name, group in formula_df.groupby('material_name'):
    arr = group.drop(['material_name'], axis=1).values
    grouped_fixed.append(arr)

X = np.array(grouped_fixed).reshape(-1, 10, 17)
X = np.nan_to_num(X, nan=0.0)
y = np.array(targets)

print(f"X.shape: {X.shape}, y.shape: {y.shape}")  # Should be (samples, 10, 17) and (samples,) for targets


X.shape: (17010, 10, 17), y.shape: (17010,)


In [10]:
X_flat = X.reshape(-1, X.shape[-1])  # Now X_flat has shape (17000 * 10, 17)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the features
X_scaled_flat = scaler.fit_transform(X_flat)

# Reshape back to the original 3D shape: (17000, 10, 17)
X_new = X_scaled_flat.reshape(X.shape)

In [13]:
X_dup = np.concatenate([X_new] * 5, axis=0)  # Shape will be (85000, 10, 17)
y_dup = np.concatenate([y] * 5, axis=0)  # Shape will be (85000,)

# Shuffle both X_dup and y_dup together while keeping alignment
X, y = shuffle(X_dup, y_dup, random_state=42)

In [ ]:
# Define CNN models
def build_cnn_c1():
    model = Sequential([
        Input(shape=(10, 17)),
        Permute((2, 1)),
        Conv1D(64, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        BatchNormalization(),
        Conv1D(64, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        MaxPooling1D(2),
        Dropout(0.4),
        Conv1D(128, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        BatchNormalization(),
        Conv1D(128, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        MaxPooling1D(2),
        Dropout(0.4),
        Conv1D(256, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        BatchNormalization(),
        Conv1D(256, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        Flatten(),
        Dense(128, activation='swish', kernel_regularizer=l2(0.0001)),
        Dropout(0.4),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

def build_cnn_c2():
    model = Sequential([
        Input(shape=(10, 17)),
        Conv1D(32, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        BatchNormalization(),
        Conv1D(32, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        MaxPooling1D(2),
        Dropout(0.4),
        Conv1D(64, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        BatchNormalization(),
        Conv1D(64, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        MaxPooling1D(2),
        Dropout(0.4),
        Conv1D(128, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        BatchNormalization(),
        Conv1D(128, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        Flatten(),
        Dense(128, activation='swish', kernel_regularizer=l2(0.0001)),
        Dropout(0.4),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

def build_cnn_c3():
    model = Sequential([
        Input(shape=(10, 17)),
        Permute((2, 1)),
        Conv1D(32, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        BatchNormalization(),
        Conv1D(32, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        MaxPooling1D(2),
        Dropout(0.4),
        Conv1D(64, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        BatchNormalization(),
        Conv1D(64, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        MaxPooling1D(2),
        Dropout(0.4),
        Conv1D(128, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        BatchNormalization(),
        Conv1D(128, 5, activation='swish', padding='same', kernel_regularizer=l2(0.0001)),
        Flatten(),
        Dense(128, activation='swish', kernel_regularizer=l2(0.0001)),
        Dropout(0.4),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

def build_lstm_r1():
    model = Sequential([
        Input(shape=(10, 17)),
        LSTM(64, return_sequences=True, kernel_regularizer=l2(0.0001)),
        LSTM(32, return_sequences=True, kernel_regularizer=l2(0.0001)),
        LSTM(32, return_sequences=False, kernel_regularizer=l2(0.0001)),
        Flatten(),
        Dense(128, activation='swish', kernel_regularizer=l2(0.0001)),
        Dropout(0.4),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Instantiate models
model_c1 = build_cnn_c1()
model_c2 = build_cnn_c2()
model_c3 = build_cnn_c3()
model_r1 = build_lstm_r1()




In [14]:
#Split data to train and test
X, X_test_final, y, y_test_final = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
#Train models
kf = KFold(n_splits=5, shuffle=True, random_state=42)
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
model_c1 = build_cnn_c1()
model_c2 = build_cnn_c2()
model_c3 = build_cnn_c3()
model_r1 = build_lstm_r1()

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Compile the model
    model_c1.compile(optimizer=Adam(learning_rate=0.001), loss='mse')  # Use 'mae' if
    # Train the model
    model_c1.fit(X_train, y_train, epochs=100, batch_size=200, validation_data=(X_test, y_test),
              callbacks=[early_stop])

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
        # Compile the model
    model_c2.compile(optimizer=Adam(learning_rate=0.001), loss='mse')  # Use 'mae' if needed
    # Train the model
    model_c2.fit(X_train, y_train, epochs=100, batch_size=200, validation_data=(X_test, y_test),
              callbacks=[early_stop])

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model_c3.compile(optimizer=Adam(learning_rate=0.001), loss='mse')  # Use 'mae' if needed
    # Train the model
    model_c3.fit(X_train, y_train, epochs=100, batch_size=200, validation_data=(X_test, y_test),
              callbacks=[early_stop])

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model_r1.compile(optimizer=Adam(learning_rate=0.001), loss='mse')  # Use 'mae' if needed
    # Train the model
    model_r1.fit(X_train, y_train, epochs=100, batch_size=200, validation_data=(X_test, y_test),
              callbacks=[early_stop])
# Save models
model_c1.save('gdrive/MyDrive/Learning/Superconductivity/cnn_c1.keras')
model_c2.save('gdrive/MyDrive/Learning/Superconductivity/cnn_c2.keras')
model_c3.save('gdrive/MyDrive/Learning/Superconductivity/cnn_c3.keras')
model_r1.save('gdrive/MyDrive/Learning/Superconductivity/lstm_r1.keras')


In [19]:
# Load models
model_c1 = load_model('gdrive/MyDrive/Learning/Superconductivity/cnn_c1.keras')
model_c2 = load_model('gdrive/MyDrive/Learning/Superconductivity/cnn_c2.keras')
model_c3 = load_model('gdrive/MyDrive/Learning/Superconductivity/cnn_c3.keras')
model_r1 = load_model('gdrive/MyDrive/Learning/Superconductivity/lstm_r1.keras')

266/266 ━━━━━━━━━━━━━━━━━━━━ 18s 62ms/step
266/266 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
266/266 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step
266/266 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
MSE for model_c1: 69.75878036217281
MSE for model_c2: 71.94456800607736
MSE for model_c3: 95.01865296367131
MSE for model_r1: 47.898437589383775


In [85]:
# Use a single test set
X_train, X_test, y_train, y_test = train_test_split(X_test_final, y_test_final, test_size=0.2, random_state=42)

# Generate predictions from base models
y_pred_c1_train = model_c1.predict(X_train).reshape(-1)
y_pred_c2_train = model_c2.predict(X_train).reshape(-1)
y_pred_r1_train = model_r1.predict(X_train).reshape(-1)

y_pred_c1_test = model_c1.predict(X_test).reshape(-1)
y_pred_c2_test = model_c2.predict(X_test).reshape(-1)
y_pred_r1_test = model_r1.predict(X_test).reshape(-1)

# Create feature matrix for Ridge Regression (stacked predictions)
X_meta_train = np.column_stack([y_pred_c1_train, y_pred_c2_train, y_pred_r1_train])
X_meta_test = np.column_stack([y_pred_c1_test, y_pred_c2_test, y_pred_r1_test])

# Train Ridge Regression as meta-learner
ridge = Ridge(alpha=1.0)
ridge.fit(X_meta_train, y_train)

# Make final predictions using the meta-learner
y_pred_ensemble = ridge.predict(X_meta_test)

# Compute MSE
mse_c1 = mean_squared_error(y_test, y_pred_c1_test)
mse_c2 = mean_squared_error(y_test, y_pred_c2_test)
mse_r1 = mean_squared_error(y_test, y_pred_r1_test)
mse_ensemble = mean_squared_error(y_test, y_pred_ensemble)

# Print results
print(f"MSE for model_c1: {mse_c1}")
print(f"MSE for model_c2: {mse_c2}")
print(f"MSE for model_r1: {mse_r1}")
print(f"MSE for Ridge ensemble: {mse_ensemble}")

213/213 ━━━━━━━━━━━━━━━━━━━━ 14s 64ms/step
213/213 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
213/213 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
MSE for model_c1: 63.523222840028524
MSE for model_c2: 68.98727246912964
MSE for model_r1: 45.27766902237446
MSE for Ridge ensemble: 45.154098155313


**Part 2: Using XGB Regressor to predict temperature based on provided material data**

In [55]:
def preprocess_data(df, target_column):
    X = df.drop(columns=[target_column])
    y = df[target_column]
    keep_cols = X.nunique()[X.nunique()/1.0 != 1.0].index.tolist()
    X = X[keep_cols]
    return train_test_split(X, y, test_size=0.2, random_state=42)

def standardize_features(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
    return X_train_scaled, X_test_scaled

In [56]:
X_train, X_test, y_train, y_test = preprocess_data(df=train, target_column='critical_temp')
X_train, X_test = standardize_features(X_train, X_test)

In [22]:
# Train XGB Model
model = XGBRegressor(n_estimators=300,subsample=0.6, reg_lambda=1,
                                reg_alpha=0.1,min_child_weight=5, max_depth=10,
                                learning_rate=0.05, gamma=0.5, colsample_bytree=0.8,
                                random_state=42)
model.fit(X_train, y_train)

# Compute SHAP Values
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

y_pred = model.predict(X_test)
name = 'XGB'
print(f"{name} MSE: {mean_squared_error(y_test, y_pred)}")
print(f"{name} R²: {r2_score(y_test, y_pred)}")

XGB MSE: 81.46099460041265
XGB R²: 0.9280445120080448


In [23]:
shap_importance = np.abs(shap_values).mean(axis=0)
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'SHAP_Importance': shap_importance})
feature_importance_df = feature_importance_df.sort_values(by="SHAP_Importance", ascending=False)
feature_importance_df.set_index('Feature', inplace=True)

In [24]:
from sklearn.metrics import mutual_info_score
from joblib import Parallel, delayed

def compute_mic_pair(X: pd.DataFrame, feature_i: str, feature_j: str, B: float):
    """
    Computes the MIC approximation for a pair of features directly from the DataFrame.

    Parameters:
    X (pd.DataFrame): Input feature matrix
    feature_i (str): First feature name
    feature_j (str): Second feature name
    B (float): Max bin limit (B = n^0.6)

    Returns:
    tuple: (feature_i, feature_j, MIC value)
    """
    if feature_i == feature_j:
        return (feature_i, feature_j, 1.0)  # Self-correlation is always 1

    # Limit bins to B = n^0.6 and ensure |X|, |Y| < B
    bins_x = min(len(np.unique(X[feature_i])), B)
    bins_y = min(len(np.unique(X[feature_j])), B)

    # Create 2D histogram
    c_xy = np.histogram2d(X[feature_i], X[feature_j], bins=[bins_x, bins_y])[0]

    # Compute Mutual Information
    mi = mutual_info_score(None, None, contingency=c_xy)

    # Normalize MIC using log2(min(|X|, |Y|))
    log_min_cardinality = np.log2(min(bins_x, bins_y) + 1)

    mic_value = mi / log_min_cardinality if log_min_cardinality > 0 else 0
    return (feature_i, feature_j, mic_value)


def mic_approx_parallel(X: pd.DataFrame, n_jobs=-1):
    """
    Computes an approximate Maximum Information Coefficient (MIC) matrix for a DataFrame
    using mutual information, with parallel processing.

    Parameters:
    X (pd.DataFrame): Input feature matrix
    n_jobs (int): Number of parallel jobs (-1 uses all available CPUs)

    Returns:
    pd.DataFrame: MIC matrix (bounded [0,1])
    """
    n_samples = X.shape[0]
    B = int(np.floor(n_samples ** 0.6))  # Compute max bins limit
    feature_names = X.columns.tolist()

    feature_pairs = [(feature_names[i], feature_names[j])
                     for i in range(len(feature_names))
                     for j in range(i, len(feature_names))]
    # Compute MIC for each pair in parallel
    results = Parallel(n_jobs=n_jobs)(
        delayed(compute_mic_pair)(X, feature_i, feature_j, B) for feature_i, feature_j in feature_pairs
    )

    # Convert results into a symmetric matrix
    mic_matrix = pd.DataFrame(np.zeros((len(feature_names), len(feature_names))),
                              index=feature_names, columns=feature_names)
    for feature_i, feature_j, mic_value in results:
        mic_matrix.loc[feature_i, feature_j] = mic_matrix.loc[feature_j, feature_i] = mic_value

    return mic_matrix

In [25]:
def distance_correlation_matrix(df: pd.DataFrame):
    """Computes the distance correlation coefficient matrix for all pairs of features in the dataframe."""
    # Initialize an empty DataFrame for the correlation matrix
    corr_matrix = pd.DataFrame(np.zeros((df.shape[1], df.shape[1])), columns=df.columns, index=df.columns)

    # Loop through each pair of columns in the DataFrame
    for col1 in df.columns:
        print(col1)
        for col2 in df.columns:
            # Calculate distance correlation coefficient for the pair (col1, col2)
            dCorr = dcor.distance_correlation(df[col1], df[col2])
            corr_matrix.loc[col1, col2] = dCorr
            corr_matrix.loc[col2, col1] = dCorr  # Since the matrix is symmetrical

    return corr_matrix

def eliminate_redundant_features(X: pd.DataFrame, mic_matrix: pd.DataFrame, dcor_matrix: pd.DataFrame, shap_importance: pd.DataFrame, threshold=0.8):
    """
    Eliminates redundant features based on MIC, Distance Correlation (Dcor), and SHAP importance.

    Parameters:
    - X (pd.DataFrame): Feature matrix
    - mic_matrix (pd.DataFrame): MIC correlation matrix
    - dcor_matrix (pd.DataFrame): Distance Correlation matrix
    - shap_importance (pd.DataFrame): DataFrame with SHAP importance (index: feature names, column: 'SHAP_Importance')
    - threshold (float): Correlation threshold for considering redundancy

    Returns:
    - pd.DataFrame: Reduced feature matrix with redundant features removed
    - list: List of removed features
    """

    # Step 1: Compute the average redundancy matrix (MIC + Dcor) / 2
    combined_matrix = (mic_matrix + dcor_matrix) / 2

    # Step 2: Identify highly correlated feature pairs
    redundant_pairs = []
    for i in range(len(combined_matrix.columns)):
        for j in range(i + 1, len(combined_matrix.columns)):
            if combined_matrix.iloc[i, j] > threshold:
                redundant_pairs.append((combined_matrix.columns[i], combined_matrix.columns[j]))

    # Step 3: Eliminate redundant features based on SHAP importance
    removed_features = set()
    for feature1, feature2 in redundant_pairs:
        if feature1 in removed_features or feature2 in removed_features:
            continue  # Skip if already removed

        # Keep the feature with the higher SHAP importance
        if shap_importance.loc[feature1, "SHAP_Importance"] >= shap_importance.loc[feature2, "SHAP_Importance"]:
            removed_features.add(feature2)
        else:
            removed_features.add(feature1)

    # Step 4: Drop redundant features and return the reduced dataset
    X_reduced = X.drop(columns=list(removed_features))

    return X_reduced, list(removed_features)


In [26]:
# Compute MIC matrix
mic_matrix_parallel = mic_approx_parallel(X_train, n_jobs=-1)
# Compute distance correlation matrix
dCorr_matrix = distance_correlation_matrix(X_train)

X_reduced, removed_features = eliminate_redundant_features(X=X_train, mic_matrix=mic_matrix_parallel, dcor_matrix=dCorr_matrix, shap_importance=feature_importance_df, threshold=0.7)

print(f"Removed Features: {removed_features}")
print(f"Remaining Features: {X_reduced.columns.tolist()}")

number_of_elements
mean_atomic_mass
wtd_mean_atomic_mass
gmean_atomic_mass
wtd_gmean_atomic_mass
entropy_atomic_mass
wtd_entropy_atomic_mass
range_atomic_mass
wtd_range_atomic_mass
std_atomic_mass
wtd_std_atomic_mass
mean_fie
wtd_mean_fie
gmean_fie
wtd_gmean_fie
entropy_fie
wtd_entropy_fie
range_fie
wtd_range_fie
std_fie
wtd_std_fie
mean_atomic_radius
wtd_mean_atomic_radius
gmean_atomic_radius
wtd_gmean_atomic_radius
entropy_atomic_radius
wtd_entropy_atomic_radius
range_atomic_radius
wtd_range_atomic_radius
std_atomic_radius
wtd_std_atomic_radius
mean_Density
wtd_mean_Density
gmean_Density
wtd_gmean_Density
entropy_Density
wtd_entropy_Density
range_Density
wtd_range_Density
std_Density
wtd_std_Density
mean_ElectronAffinity
wtd_mean_ElectronAffinity
gmean_ElectronAffinity
wtd_gmean_ElectronAffinity
entropy_ElectronAffinity
wtd_entropy_ElectronAffinity
range_ElectronAffinity
wtd_range_ElectronAffinity
std_ElectronAffinity
wtd_std_ElectronAffinity
mean_FusionHeat
wtd_mean_FusionHeat
gmean

In [28]:
from deap import base, creator, tools, algorithms  # Genetic Algorithm library
import random

# ---------------- Step 1: Define Fitness Function ---------------- #
def evaluate_feature_subset(individual, X, y):
    """ Evaluates a given feature subset (individual) using CatBoost & cross-validation """

    # Convert binary feature selection into actual column names
    selected_features = [feature for i, feature in enumerate(X.columns) if individual[i] == 1]

    # If no features selected, return worst possible score
    if len(selected_features) == 0:
        return (-float("inf"),)  # Low R² is bad

    # Train model only on selected features
    model = CatBoostRegressor(iterations=100, depth=6, learning_rate=0.1, verbose=0, random_seed=42)
    r2 = np.mean(cross_val_score(model, X[selected_features], y, cv=3, scoring='r2'))  # ✅ Maximize R²

    return (r2,)  # GA maximizes R²

# ---------------- Step 2: Set Up Genetic Algorithm ---------------- #
def run_genetic_algorithm(X, y, population_size=50, generations=20, crossover_prob=0.8, mutation_prob=0.2):
    """
    Runs a Genetic Algorithm to find the optimal subset of features for regression.
    """

    num_features = X.shape[1]

    # Define optimization problem: Maximize R²
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # ✅ Maximizing R²
    creator.create("Individual", list, fitness=creator.FitnessMax)

    # Define Individual and Population
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)  # Binary selection (0=off, 1=on)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, num_features)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Register genetic operations
    toolbox.register("mate", tools.cxTwoPoint)  # Crossover
    toolbox.register("mutate", tools.mutFlipBit, indpb=mutation_prob)  # Mutation
    toolbox.register("select", tools.selTournament, tournsize=3)  # Selection
    toolbox.register("evaluate", evaluate_feature_subset, X=X, y=y)

    # Initialize population
    population = toolbox.population(n=population_size)

    # Run GA
    result_population, _ = algorithms.eaSimple(population, toolbox, cxpb=crossover_prob, mutpb=mutation_prob,
                                               ngen=generations, verbose=True)

    # Get best individual (best feature subset)
    best_individual = tools.selBest(result_population, k=1)[0]
    best_features = [feature for i, feature in enumerate(X.columns) if best_individual[i] == 1]

    return best_features

In [29]:
reduced_features = [i for i in feature_importance_df[feature_importance_df['SHAP_Importance'] != 0].index.tolist() if i not in removed_features]
X_original = X_reduced[reduced_features]

best_features = run_genetic_algorithm(X_original, y_train, population_size=20, generations=5)

print(f"Best Features Selected by GA: {best_features}")

gen	nevals
0  	20    
1  	16    
2  	18    
3  	14    
4  	14    
5  	18    
Best Features Selected by GA: ['range_ThermalConductivity', 'wtd_gmean_ThermalConductivity', 'range_atomic_radius', 'wtd_gmean_Valence', 'wtd_std_ThermalConductivity', 'wtd_std_ElectronAffinity', 'mean_Density', 'wtd_entropy_ThermalConductivity', 'gmean_Density', 'wtd_std_Valence', 'std_Density', 'wtd_range_fie', 'std_atomic_radius', 'wtd_range_atomic_mass', 'wtd_range_ElectronAffinity', 'wtd_entropy_atomic_mass', 'wtd_entropy_Density', 'wtd_std_atomic_mass', 'wtd_std_Density', 'wtd_entropy_FusionHeat', 'entropy_atomic_mass', 'wtd_entropy_ElectronAffinity', 'entropy_ThermalConductivity', 'wtd_mean_ElectronAffinity', 'entropy_atomic_radius', 'wtd_gmean_Density', 'wtd_mean_Density', 'wtd_range_FusionHeat', 'wtd_std_FusionHeat', 'wtd_mean_fie', 'wtd_gmean_fie', 'wtd_range_Density', 'wtd_gmean_FusionHeat', 'wtd_range_ThermalConductivity', 'gmean_fie', 'mean_fie', 'mean_atomic_radius', 'gmean_atomic_radius', 'mean_

In [80]:
X = X_reduced[best_features].values
y = y_train.values

X_dup = np.concatenate([X] * 5, axis=0)
y_dup = np.concatenate([y] * 5, axis=0)

# Shuffle both X_dup and y_dup together while keeping alignment
X, y = shuffle(X_dup, y_dup, random_state=42)

In [81]:
# Define KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)



# Store results
mse_scores = []
y_preds = []

# Perform K-Fold training
for fold, (train_index, test_index) in enumerate(kf.split(X), start=1):
    X_train2, X_test2 = X[train_index], X[test_index]
    y_train2, y_test2 = y[train_index], y[test_index]

    # **New Step: Split training into actual train and validation**
    X_train_final, X_val, y_train_final, y_val = train_test_split(
        X_train2, y_train2, test_size=0.2, random_state=42
    )
    # Initialize model
    model = XGBRegressor(n_estimators=100, subsample=0.6, reg_lambda=3,
                      reg_alpha=0.1, min_child_weight=5, max_depth=10,
                      learning_rate=0.05, gamma=0.5, colsample_bytree=0.8,
                      random_state=42)
    # Train the model with early stopping
    model.fit(
        X_train_final, y_train_final,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # Predict on the test set (never seen before)
    y_pred = model.predict(X_test2)
    y_preds.append(y_pred)

    # Compute MSE for this fold
    mse = mean_squared_error(y_test2, y_pred)
    mse_scores.append(mse)

    print(f"Fold {fold}: MSE = {mse:.4f}")



Fold 1: MSE = 37.5209
Fold 2: MSE = 40.2129
Fold 3: MSE = 43.1075
Fold 4: MSE = 38.4854
Fold 5: MSE = 40.5744


In [82]:

y_pred = model.predict(X_test[best_features].values)
name = 'XGB Final'
print(f"{name} MSE: {mean_squared_error(y_test.values, y_pred)}")
print(f"{name} R²: {r2_score(y_test.values, y_pred)}")

XGB Final MSE: 83.71458147143977
XGB Final R²: 0.9264266786604417


In [83]:
import xgboost as xgb
# Convert data to DMatrix (XGBoost's optimized data structure)
dtrain = xgb.DMatrix(X, label=y)

# Define XGBoost parameters
params = {
    "objective": "reg:squarederror",
    "n_estimators": 100,
    "subsample": 0.6,
    "reg_lambda": 1,
    "reg_alpha": 0.1,
    "min_child_weight": 5,
    "max_depth": 10,
    "learning_rate": 0.05,
    "gamma": 0.5,
    "colsample_bytree": 0.8,
    "random_state": 42,
}

# Perform cross-validation
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=300,  # Equivalent to n_estimators
    nfold=5,  # 5-fold cross-validation
    metrics="rmse",
    early_stopping_rounds=10,  # Stops training if validation error doesn't improve
    as_pandas=True,  # Get results as a DataFrame
    seed=42,
)

# Print final results
print(cv_results.tail(5))  # Shows last few rounds of training

# Extract best RMSE score
best_rmse = cv_results["test-rmse-mean"].min()
print(f"Best Test RMSE: {best_rmse:.4f}")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:47:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:47:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:47:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


     train-rmse-mean  train-rmse-std  test-rmse-mean  test-rmse-std
295         4.366397        0.030927        4.784158       0.131464
296         4.364650        0.031152        4.781549       0.131417
297         4.363446        0.031115        4.780251       0.132616
298         4.362175        0.031363        4.779473       0.132225
299         4.360342        0.031068        4.777819       0.133022
Best Test RMSE: 4.7778


In [84]:
best_rounds = cv_results.shape[0]
print(f"Best number of boosting rounds: {best_rounds}")

# Train final model using best_rounds
final_model = xgb.train(params, dtrain, num_boost_round=best_rounds)

# Test on new data
dtest = xgb.DMatrix(X_test[best_features].values)
y_pred = final_model.predict(dtest)

# Compute test MSE
test_mse = mean_squared_error(y_test.values, y_pred)
print(f"Test MSE: {test_mse:.4f}")

Best number of boosting rounds: 300


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:49:43] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


Test MSE: 83.5239


**Part 3: Predictions**

In [89]:
material_elements = formula_test.drop(columns=["material"])
material_elements = material_elements.astype(float)
# Initialize a list to store merged data
merged_data = []

# Iterate through each material in material_elements with a progress bar
for material_index, material_row in tqdm(material_elements.iterrows(), total=len(material_elements), desc="Processing materials"):
    material_name = material_row.name  # Material is the index (name) of the row

    # Get the non-zero element counts for this material
    elements_in_material = material_row[material_row > 0].index.tolist()  # List of elements in material with non-zero counts

    # Create a temporary DataFrame for the current material
    material_data = []

    for i in range(10):  # Maximum 10 rows per material
        if i < len(elements_in_material):
            element = elements_in_material[i]
            element_info = element_features[element_features['Element'] == element].copy()
            # Add element count (how many times this element occurs) to the feature data
            element_info['count'] = material_row[element]
        else:
            # Fill with zeros for padding
            element_info = pd.DataFrame(np.zeros((1, len(element_features.columns) + 1)),
                                        columns=element_features.columns.tolist() + ['count'])

        # Append the element's features to the material data
        material_data.append(element_info)

    # Combine material data into a single DataFrame for this material
    material_df = pd.concat(material_data, ignore_index=True)
    material_df['material_name'] = material_name  # Add the material name as a column

    # Append the material's data to the merged dataset
    merged_data.append(material_df)

Processing materials: 100%|██████████| 4253/4253 [00:33<00:00, 126.47it/s]


In [111]:
# Combine all material data into a single DataFrame
features = ['count','AtomicMass','AtomicNumber','FirstIonizationEnergy','AtomicRadius','BoilingPoint',
            'Density','ElectronAffinity','Electronegativity','FusionHeat','MeltingPoint','SpecificHeat',
            'ThermalConductivity','ThermalExpansion','Valence','MolarVolume','Period', 'material_name'
]
final_df = pd.concat(merged_data, ignore_index=True)
formula_df = final_df[features]

# Group by material_name to get arrays for CNN
grouped_fixed = []
for material_name, group in formula_df.groupby('material_name'):
    arr = group.drop(['material_name'], axis=1).values
    grouped_fixed.append(arr)

X = np.array(grouped_fixed).reshape(-1, 10, 17)
X = np.nan_to_num(X, nan=0.0)

print(f"X.shape: {X.shape}")

X.shape: (4253, 10, 17)


In [112]:
X_flat = X.reshape(-1, X.shape[-1])  # Now X_flat has shape (17000 * 10, 17)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the features
X_scaled_flat = scaler.fit_transform(X_flat)

# Reshape back to the original 3D shape: (17000, 10, 17)
X = X_scaled_flat.reshape(X.shape)

In [92]:
y_pred_c1_test = model_c1.predict(X).reshape(-1)
y_pred_c2_test = model_c2.predict(X).reshape(-1)
y_pred_r1_test = model_r1.predict(X).reshape(-1)

X_meta_test = np.column_stack([y_pred_c1_test, y_pred_c2_test, y_pred_r1_test])

# Make final predictions using the meta-learner
y_pred_ensemble = ridge.predict(X_meta_test)


133/133 ━━━━━━━━━━━━━━━━━━━━ 13s 100ms/step
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [103]:
output = pd.DataFrame(y_pred_ensemble)
output.columns = ['critical_temp']
output.to_csv('gdrive/MyDrive/Learning/Superconductivity/predictions.csv', index_label = 'index')

In [106]:
output = pd.DataFrame(y_pred_r1_test)
output.columns = ['critical_temp']
output.to_csv('gdrive/MyDrive/Learning/Superconductivity/predictions_r1.csv', index_label = 'index')

In [107]:
output = pd.DataFrame(y_pred_c1_test)
output.columns = ['critical_temp']
output.to_csv('gdrive/MyDrive/Learning/Superconductivity/predictions_c1.csv', index_label = 'index')

In [108]:
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(test), columns=test.columns)

y_pred_xgb = model.predict(X[best_features].values)


In [110]:
output = pd.DataFrame(y_pred_xgb)
output.columns = ['critical_temp']
output.to_csv('gdrive/MyDrive/Learning/Superconductivity/predictions_xgb.csv', index_label = 'index')